**Mounting google drive on colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**listing all the images present in the folder**

In [ ]:
import os

# List files in the 'meme_images' folder in Google Drive
folder_path = '/content/drive/MyDrive/meme_images'
files = os.listdir(folder_path)
print(files)


['Copy of 2ivkuv.jpg', 'Copy of 2 meme.jpeg', 'Copy of 2dfi31.jpg', 'Copy of 1ihzfe.jpg', 'Copy of 2gnnjh.jpg', 'Copy of 1e7ql7.jpg', 'Copy of 1oyap8.jpg', 'Copy of 1meme.jpeg', 'Copy of 4fyydx.jpg', 'Copy of 4og03c.jpg', 'Copy of animal.jpeg', 'Copy of confused.jpeg', 'Copy of car (1).jpg', 'Copy of bus.jpg', 'Copy of 7 meme.jpeg', 'Copy of bird.jpg', 'Copy of city.jpg', 'Copy of angry.jpg', 'Copy of celebrate.jpg', 'Copy of car.jpg', 'Copy of download.jpeg', 'Copy of hardwork.jpg', 'Copy of nature.jpg', 'Copy of cycle.jpg', 'Copy of confused1.jpeg', 'Copy of food (1).jpg', 'Copy of fire.jpg', 'Copy of food.jpg', 'Copy of happy.jpg', 'Copy of happy1.jpg', 'Copy of smile1.jpg', 'Copy of serious.jpg', 'Copy of river2.jpg', 'Copy of smile2.jpg', 'Copy of smile.jpg', 'Copy of sad.jpg', 'Copy of rain.jpg', 'Copy of river.jpg', 'Copy of smile3.png', 'Copy of sarcastic.jpeg', 'Copy of tired.jpeg', 'Copy of tear.jpg', 'Copy of space.jpg', 'Copy of tired.jpg', 'Copy of smile4.jpeg', 'Copy of s

**IMPORTS**

**1. Importing Libraries**

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import textwrap
import random
import os
import ipywidgets as widgets
from IPython.display import display
from transformers import GPT2Tokenizer, GPT2LMHeadModel


**Load GPT-2*8

In [ ]:
# Load GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**AI Based Captions**

In [ ]:
def generate_caption(prompt, emotion=None):
    # If an emotion is provided, prepend it to the prompt
    if emotion:
        prompt = f"{emotion.capitalize()} caption: {prompt}"

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=30,
        num_return_sequences=1,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    caption = tokenizer.decode(output[0], skip_special_tokens=True)
    caption = caption[len(prompt):].strip()
    max_length = 100
    if len(caption) > max_length:
        caption = caption[:max_length] + '...'
    return caption



**Adding Captions to Images**

In [ ]:
def add_caption_to_image(image, caption, position, font, font_size, font_color):
    draw = ImageDraw.Draw(image)
    try:
        # Load a standard font
        font = ImageFont.truetype(font, font_size)
    except IOError:
        # Fall back to default font if the specified font cannot be loaded
        font = ImageFont.load_default()
    wrapped_text = textwrap.fill(caption, width=40)
    textwidth, textheight = draw.textsize(wrapped_text, font)
    width, height = image.size

    if position == 'top':
        x = (width - textwidth) / 2
        y = 10
    elif position == 'bottom':
        x = (width - textwidth) / 2
        y = height - textheight - 10
    elif position == 'left':
        x = 10
        y = (height - textheight) / 2
    elif position == 'right':
        x = width - textwidth - 10
        y = (height - textheight) / 2
    else:
        x = (width - textwidth) / 2
        y = height - textheight - 10

    draw.text((x, y), wrapped_text, font=font, fill=font_color)
    return image


**Selecting Images**

In [ ]:
def select_image_by_keyword(keyword):
    keyword_to_images = {
        "animal": ["animal.jpeg"],
        "city": ["city.jpg"],
        "fire": ["fire.jpg"],
        "river": ["river.jpg", "river2.jpg"],
        "car": ["car.jpg", "car (1).jpg"],
        "smile": ["smile.jpg", "smile1.jpg", "smile2.jpg", "smile3.jpg", "smile4.jpeg"],
        "happy": ["happy.jpg", "happy1.jpg"],
        "space": ["space.jpg"],
        "rain": ["rain.jpg"],
        "tear": ["tear.jpg"],
        "confused": ["confused.jpeg", "confused1.jpeg"],
        "hardwork": ["hardwork.jpg"],
        "food": ["food.jpg", "food (1).jpg"],
        "serious": ["serious.jpg"],
        "celebrate": ["celebrate.jpg"],
        "sun": ["sun.jpg"],
        "cycle": ["cycle.jpg"],
        "sad": ["sad.jpg"],
        "bird": ["bird.jpg"]
    }

    if keyword in keyword_to_images:
        image_files = keyword_to_images[keyword]
        if image_files:
            for image_file in image_files:
                full_image_path = os.path.join(meme_images_path, image_file)
                if os.path.exists(full_image_path):
                    return full_image_path
    return None

def select_random_image():
    image_files = os.listdir(meme_images_path)
    if not image_files:
        raise ValueError("No images found in the folder.")
    image_file = random.choice(image_files)
    return os.path.join(meme_images_path, image_file)


**Updating our meme**

In [ ]:
def update_meme(caption, position, font, font_size, font_color, keyword):
    image_path = select_image_by_keyword(keyword) or select_random_image()
    image = Image.open(image_path)
    image_with_caption = add_caption_to_image(image, caption, position, font, font_size, font_color)
    plt.imshow(image_with_caption)
    plt.axis('off')
    plt.show()


**Creating basic UI**

In [ ]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

keyword_to_images = {
    "animal": ["animal.jpeg"],
    "city": ["city.jpg"],
    "fire": ["fire.jpg"],
    "river": ["river.jpg", "river2.jpg"],
    "car": ["car.jpg", "car (1).jpg"],
    "smile": ["smile.jpg", "smile1.jpg", "smile2.jpg", "smile3.jpg", "smile4.jpeg"],
    "happy": ["happy.jpg", "happy1.jpg"],
    "space": ["space.jpg"],
    "rain": ["rain.jpg"],
    "tear": ["tear.jpg"],
    "confused": ["confused.jpeg", "confused1.jpeg"],
    "hardwork": ["hardwork.jpg"],
    "food": ["food.jpg", "food (1).jpg"],
    "serious": ["serious.jpg"],
    "celebrate": ["celebrate.jpg"],
    "sun": ["sun.jpg"],
    "cycle": ["cycle.jpg"],
    "sad": ["sad.jpg"],
    "bird": ["bird.jpg"]
}

caption_input = widgets.Text(value='This is a sample caption', description='Caption:')
caption_type = widgets.RadioButtons(options=['Manual', 'AI'], description='Caption Type:')
position_input = widgets.Dropdown(options=['top', 'bottom', 'left', 'right'], value='bottom', description='Position:')
font_input = widgets.Text(value='DejaVuSans.ttf', description='Font:', placeholder='Font file path')
font_size_input = widgets.IntSlider(value=20, min=10, max=100, step=5, description='Font Size:')
font_color_input = widgets.ColorPicker(value='white', description='Font Color:')
keyword_input = widgets.Dropdown(
    options=list(keyword_to_images.keys()), # keyword_to_images is now defined in the scope
    value='animal',
    description='Keyword:'

)

In [ ]:
emotion_input = widgets.Dropdown(
    options=['None', 'Happy', 'Sad', 'Stressed', 'Funny'],
    value='None',
    description='Emotion:',
)


**UI Logic**

In [ ]:
def handle_caption_type(change):
    if change['new'] == 'AI':
        ai_prompt_widget.value = ''
        caption_input.disabled = True
    else:
        ai_prompt_widget.value = ''
        caption_input.disabled = False

caption_type.observe(handle_caption_type, names='value')

def handle_ai_prompt(change):
    if caption_type.value == 'AI' and change['new']:
        caption_prompt = change['new']
        selected_emotion = emotion_input.value
        caption_input.value = generate_caption(caption_prompt, emotion=selected_emotion)



**Displaying the output**

In [ ]:
# Path to the meme_images folder
meme_images_path = '/content/drive/MyDrive/meme_images'

# Assuming ai_prompt_widget is a text input widget, define it here
ai_prompt_widget = widgets.Text(
    value='',
    description='AI Prompt:',
    disabled=False
)

# Display widgets
ui = widgets.VBox([
    caption_type,
    caption_input,
    ai_prompt_widget, # Now ai_prompt_widget is defined
    emotion_input,  # Added emotion selection here
    keyword_input,
    position_input,
    font_input,
    font_size_input,
    font_color_input
])
out = widgets.interactive_output(update_meme, {
    'caption': caption_input,
    'position': position_input,
    'font': font_input,
    'font_size': font_size_input,
    'font_color': font_color_input,
    'keyword': keyword_input
})

display(ui, out)

Output()